In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl
import random

In [ ]:
!pip install fastai --upgrade

In [ ]:
from fastai.tabular.all import *

In [ ]:
# To avoid loading a 16GB datafile I randomly sample 50.000 rows from the train data
# Change s if you want more rows :)

In [ ]:
random.seed(1) #fix random seed for consistency

In [ ]:
%%time
filename = "../input/amex-default-prediction/train_data.csv"
n = 5531451 #sum(1 for line in open(filename)) - 1 --> code to get n the first time. Number of records in file (excludes header)
s = 50000 #desired sample size
skip = sorted(random.sample(range(1,n+1),n-s)) #the 0-indexed header will not be included in the skip list
df = pd.read_csv(filename, skiprows=skip)
df.head()

In [ ]:
filename = "../input/amex-default-prediction/train_labels.csv"
df_y = pd.read_csv(filename, skiprows=skip)
df_y.head()

In [ ]:
train_df = df.join(df_y.set_index('customer_ID'),on='customer_ID')

In [ ]:
train_df.head()

In [ ]:
train_df.info(verbose=True, show_counts =True)

In [ ]:
train_df = add_datepart(train_df, 'S_2')

In [ ]:
train_df.to_csv('sample_train.csv')

In [ ]:
cat_columns = ['B_30',
 'B_38',
 'D_114',
 'D_116',
 'D_117',
 'D_120',
 'D_126',
 'D_66',
'customer_ID',
'D_64',
'D_63',
 'D_68', 'S_2Day', 'S_2Dayofweek', 'S_2Dayofyear', 'S_2Elapsed',
       'S_2Is_month_end', 'S_2Is_month_start', 'S_2Is_quarter_end',
       'S_2Is_quarter_start', 'S_2Is_year_end', 'S_2Is_year_start',
       'S_2Month', 'S_2Week', 'S_2Year']

In [ ]:
columns = train_df.columns.tolist()
cont_columns = [column for column in columns if column not in cat_columns]
cont_columns.remove('target')

In [ ]:
train_df[cat_columns].info()

In [ ]:
a = train_df['customer_ID'].unique()
valid = np.random.choice(a,replace=False, size=10000)
len(pd.unique(train_df['customer_ID'])), len(valid)

#splits = (list(train_idx),list(valid_idx))

In [ ]:
valid_idx = train_df.index[train_df['customer_ID'].isin(valid)]
train_idx = train_df.index[train_df['customer_ID'].isin(valid)==False]

splits = (list(train_idx),list(valid_idx))

In [ ]:
# DATA LOADER -- For Pytorch/fast.ai tabular learners
"""
dls = TabularDataLoaders.from_csv('./sample_train.csv', y_names="target",
    cat_names = cat_columns,
    cont_names = cont_columns,
    splits=splits,
    procs=[Categorify, FillMissing])

len(dls.train),len(dls.valid)
"""

In [ ]:
# Tabular pandas -- For fast.ai/sklearn implementation

to = TabularPandas(train_df, procs=[Categorify, FillMissing],
                   cat_names = cat_columns,
                   cont_names = cont_columns,
                   y_names='target',
                   splits=splits)

len(to.train),len(to.valid)

In [ ]:
xs,y = to.train.xs,to.train.y
valid_xs,valid_y = to.valid.xs,to.valid.y

In [ ]:
!pip install dtreeviz --upgrade

In [ ]:
to.show(10)

In [ ]:
to['target'].isna().sum()
train_df['target'].isna().sum()

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from dtreeviz.trees import *
#from IPython.display import Image, display_svg, SVG

pd.options.display.max_rows = 150
pd.options.display.max_columns = 150

classifier = DecisionTreeClassifier(max_leaf_nodes=4)
classifier.fit(xs, y);

In [ ]:
xs.columns

In [ ]:
viz = dtreeviz(classifier, 
               xs, 
               y,
               target_name='default',
               feature_names=['No','Yes'], 
               class_names=xs.columns.tolist()  # need class_names for classifier
              )  
              
viz.view() 